In [ ]:
from opty import Problem, create_objective_function, parse_free
import sympy as sp
import numpy as np
import scipy as sc
import time as tm
import pickle
import sympy.physics.mechanics as me
import sys
sys.path.insert(0, "..")
from importlib import reload
import matplotlib.pyplot as plt

initPos = 'InitPosOptQuat'

motion_folder_list = ['Abduction_rigged2','Elevation','Abduction_rigged','Abduction','Steering']
motion_list = ['abd_rigged.mat','elevation.mat','abd_rigged.mat','abduction.mat','steering.mat']
weights_list = [100,150,200,250,300]

for i in range(len(motion_folder_list)):
    for iweight in range(len(weights_list)):
        motion_folder = motion_folder_list[i]
        motion_name = motion_list[i]
        struct_name = 'results_quat_QuatInit_'+str(weights_list[iweight])
        act_w = 1
        traj_w = weights_list[iweight]
        vel_w = 0.1

        model_struct = sc.io.loadmat('../Motions/'+motion_folder+'/OS_model.mat')
        data_struct = sc.io.loadmat('../data_model.mat')
        start = tm.time()
        MM,FO,q,w,u0,fr,frstar,kinematical,xdot,holonomic = eq.create_eoms_u0state(model_struct,data_struct,initPos,derive = 'numeric')
        TE,activations = eq.polynomials_quat(model_struct = model_struct,q = q,derive = 'numeric',model_params_struct = data_struct ,initCond_name = initPos)
        # q = eq.create_eoms_u0state(model_struct,data_struct,initPos,derive = 'numeric')

        time_to_create = tm.time() - start
        print(time_to_create)
        import equations as eq
        reload (eq);
        # dict_vals,symlist, value_list = eq.create_parameters_dict(data_struct, initPos)
        x0 = data_struct['params'][initPos][0,0]['initCondQuat'].item()
        x0t = list(x0.T[0])
        eoms_implicit = sp.Matrix(kinematical).col_join(fr+frstar+TE).col_join(holonomic)
        import trajectory_lib as tr
        reload (tr);
        num_nodes = 101
        file = '../Motions/' + motion_folder + '/' + motion_name
        traj_original, interval_value, time = tr.exp_trajectory_quat(file,num_nodes)
        traj = tr.exp_trajectory_quat_2_myobj(traj_original)

        state_symbols = tuple(q+w+u0)
        num_states = len(state_symbols)
        specified_symbols = tuple(activations)
        num_inputs = len(specified_symbols)
        t = me.dynamicsymbols._t
        objective_traj,objective_traj_jac = eq.custom_objective_quat(len(q),interval_value)

        def obj(free):
            # min_traj = traj_w * interval_value * np.sum((traj - free[:13*num_nodes])**2)
            min_traj = traj_w * np.sum(objective_traj(np.split(free[:13*num_nodes],13),traj))
            min_vel = vel_w * interval_value * np.sum((free[13*num_nodes:num_states*num_nodes])**2)
            min_torque = act_w * interval_value * np.sum(free[num_states*num_nodes:(num_states + num_inputs)*num_nodes]**2)
            return min_traj + min_torque + min_vel

        def obj_grad(free):
            grad = np.zeros_like(free)
            # grad[:13*num_nodes] = traj_w * 2.0 * interval_value * (free[:13*num_nodes] - traj)
            grad[:13*num_nodes] = traj_w * np.concatenate(objective_traj_jac(np.split(free[:13*num_nodes],13),traj))
            grad[13*num_nodes:num_states*num_nodes] = vel_w * 2 * interval_value * free[13*num_nodes:num_states*num_nodes]
            grad[num_states*num_nodes:(num_states + num_inputs)*num_nodes] = act_w * 2.0 * interval_value * free[num_states*num_nodes:(num_states + num_inputs)*num_nodes]
            return grad
        instance_constraints = []
        # for i in range(13):
        instance_constraints.append(state_symbols[-3].func(0.0)-0) 
            
        bounds1 = (0.0,1.0)
        bounds = (bounds1,)*len(activations)
        bndrs = dict(zip(activations,bounds))
        start = tm.time()
        prob = Problem(obj, obj_grad, eoms_implicit, state_symbols,
                    num_nodes, interval_value,
                    known_parameter_map={},
                    instance_constraints=instance_constraints,
                    bounds=bndrs,
                    integration_method='midpoint')


        time_to_create = tm.time() - start
        print(time_to_create)



        prob.add_option('max_iter',3000)
        prob.add_option('limited_memory_max_history', 40)
        initial_guess = np.zeros(prob.num_free)
        initial_guess[:13*num_nodes] = traj_original.flatten()
        time_2_solve_start = tm.time()
        solution, info = prob.solve(initial_guess)
        time_2_solve = tm.time() - time_2_solve_start
        print(info['status_msg'])
        print(info['obj_val'])
        act_obj = np.sum(solution[num_states*num_nodes:(num_states + num_inputs)*num_nodes]**2)
        print('Objective activations: ', act_obj)
        reload (tr);
        import matplotlib.pyplot as plt
        tr.plot_results(solution,traj,time,num_nodes)
        # import matplotlib.pyplot as plt
        # fig, axes = plt.subplots(int(num_states+num_inputs), 1, sharex=True,
        #                          figsize=(6.4, 0.8*(num_states+num_inputs)),
        #                          layout='compressed')
        # prob.plot_trajectories(solution, axes=axes)
        import matplotlib.pyplot as plt
        fig, axs = plt.subplots(13)
        for j in range(13):
            axs[j].plot(time,traj[j,:])
            axs[j].plot(time,solution[j*num_nodes:(j+1)*num_nodes])
            fig.set_figheight(10)
        import trajectory_lib as tr
        reload (tr);
        # num_iter_sol = int(input('enter number of iterarions:'))
        file_name = '../Motions/'+motion_folder+'/' + struct_name + '.mat'
        tr.sol2struct(solution,activations,len(q),num_states,num_nodes,time,0,time_2_solve,file_name)

        file_name_mot = '../Motions/'+motion_folder+'/' + struct_name + '.mot'
        tr.sol2mot_quat(solution, num_nodes, len(q), time, file_name_mot)


KeyboardInterrupt: 